In [1]:
%pylab inline
%load_ext autoreload

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pn_analyze

In [3]:
import cProfile

In [4]:
import os
import time
import socket
import sqlite3
import datetime

from collections import OrderedDict,namedtuple

import bokeh
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine

from bokeh.plotting import figure, output_notebook, show
matplotlib.style.use('ggplot')

In [5]:
import pn_analyze, _data, _graphs

In [6]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.charts import Bar, output_file, show
import bokeh.charts as chart

In [7]:
DB_FILE_LOCATION='/home/jshank/ext/code/pynet_database.db'
now = datetime.datetime.now()
REGEX_PATTERN_GCI =  r'[A-Z]\w{5,7}'
TIMESTAMP_PARSE_DICT = {'case_timestamp': '%Y-%m-%dT%H:%M:%S.%fZ'}


In [8]:
output_notebook()

Loading BokehJS ...

In [9]:
disk_engine = create_engine('sqlite:///%s' % DB_FILE_LOCATION)
def loadup_func(disk_engine):
    metrics = pn_analyze.TestResult(disk_engine, wait_interval=50, writing_to_csv=False)
    metrics.strict_startup()
    return metrics

In [ ]:
b = time.time()
metrics = loadup_func(disk_engine)
print(time.time() - b)

In [ ]:
latest = dis_year[dis_year.group_uuid.isin([_data.return_guuid_latest(dis_year)])]

In [ ]:
len(metrics.df[metrics.df.group_uuid == '16d5322813624f8e8449ef803b69bbab'].case_id.unique())

In [ ]:
len(metrics.df[metrics.df.group_uuid == 'fe0d174bb5544e669141a048c6e7e164'])

In [ ]:
latest[_data.return_in_norm_df(latest, 'case_duration', 2)]

In [ ]:
met

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
metrics.df[metrics.this_month].case_action.value_counts().plot(kind='bar', ax=ax, color='gray')
ax.set_title('Coverage by Endpoint')

In [ ]:
from bokeh.charts import Histogram, output_file, show
from bokeh.sampledata.autompg import autompg as df
import bokeh.plotting as bk

hist = Histogram(df, values='hp', color='cyl',
                 title="HP Distribution by Cylinder Count", legend='top_right')


In [ ]:
h = chart.Histogram(latest.case_action.map(hist_map), bins = 2*len(hist_map))
h.xaxis[0].formatter = FixedTickFormatter(labels=inv_map)
show(h)


In [ ]:

pf_metrics = metrics.df[(metrics.df.numeric_status !=3) & metrics.this_month]


In [ ]:
caseidgroup = pf_metrics.groupby('case_id')
ssorted = caseidgroup.numeric_status.mean()
ssorted.sort_values()[:10]


In [ ]:
def highest_failures_by_groupby_count(groupby_df, count):
    mean_status = groupby_df.numeric_status.mean()
    worst = mean_status.sort_values()[:count]
    return worst

def highest_failures_by_groupby_stdev(groupby_df, sigma=0):
    vals = groupby_df.numeric_status.mean()
    normed_vals = vals[~_data.return_in_norm_series(vals, sigma)]
    not_pass_fail_100 = normed_vals[(normed_vals <= 0.95) & (normed_vals != 0)]
    print(type(not_pass_fail_100))
    return not_pass_fail_100

In [ ]:
def bar_plot_w_numeric_status(groupby_df, sigma=0):
    fig, ax = plt.subplots(figsize=(20,10))
    not_pass_fail_100 = highest_failures_by_groupby_stdev(groupby_df, sigma)
    ax.bar(np.arange(len(not_pass_fail_100)), not_pass_fail_100)
    ax.set_xticks(np.arange(len(not_pass_fail_100))+0.5)
    ax.set_xticklabels(not_pass_fail_100.keys(), rotation=80)
    return fig, ax, not_pass_fail_100.keys()

In [ ]:
fig, ax, list_keys = bar_plot_w_numeric_status(pf_metrics.groupby('case_id'), 1.8)
ax.set_title('Highest % of Failures by Test ID')


In [ ]:
hardy = pf_metrics[(pf_metrics.numeric_status == 0) | (pf_metrics.numeric_status == 1)].groupby('case_id').numeric_status.mean()

In [ ]:
p = Bar(highest_failures_by_groupby_count(metrics.df.groupby('case_id'), 20), plot_width=800, plot_height=500, legend=None, 
        title='Lowest 20 Passing Rate Case Ids for Current Month', ylabel='Pass Rate')

In [ ]:
p = Bar(highest_failures_by_groupby_stdev(pf_metrics.groupby('case_id'), 1.5))
show(p)

In [ ]:
len(metrics.df.case_description.unique())

In [ ]:
boop = highest_failures_by_groupby(pf_metrics.groupby('case_id'), 1.5)


In [ ]:

aggregation = {
    'case_duration' : {
        'mean_duration': 'mean',
        'std_dev': 'std'
    }
}
groupby_id = metrics.df.groupby('case_id')
mean_duration_df = groupby_id.agg(aggregation).dropna()
max_duration= mean_duration_df[(mean_duration_df.case_duration.mean_duration == 
                                                   mean_duration_df.case_duration.mean_duration.max())]
max_duration_over_time = _data.return_specific_case_over_time(metrics.df, max_duration.index.values[0])

In [ ]:
max_duration_over_time.head()

In [ ]:
def plot(df, fig, ax):
    if df.case_client_ip.unique() == 1:
        case_client_ip = df.case_client_ip.unique()[0]
        try:
            hostname = get_hostname(case_client_ip)
        except:
            hostname = case_client_ip
    else:
        hostname = None
    df.plot('date_int', 'case_duration', kind='scatter', ax=ax, color=np.random.rand(3,1), label=hostname)
#     xtick_range = (over_time.dropna().date_int.max() - over_time.dropna().date_int.min()) / 10
#     current_xticks = [a.get_text() for a in ax.get_xticklabels()]
    d = list(map(lambda x: pd.to_datetime(x).strftime('%m-%d-%Y'), ax.get_xticks()))
    ax.set_xticklabels(d, rotation=70)
    ax.set_title('.'.join(df.case_id.unique()))

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plot(_data.return_longest_case_over_time(metrics.df[(metrics.df.case_endpoint.str.contains('localhost')) & 
                                            (metrics.df.case_client_ip == '10.1.178.183') & (metrics.df.passed == 1)],
                                   aggregation), fig, ax)


In [ ]:
a = _data.return_longest_case_over_time(metrics.df[metrics.df.case_endpoint.str.contains('qacombo016')], aggregation)
b = _data.return_in_norm_df(a, 'case_duration', 1)
len(a[b])
a = cm.get_cmap('nipy_spectral')

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
t = 0
for ip in metrics.df.case_client_ip.unique():
    if ip:
#         print(ip)
        plot(_data.return_longest_case_over_time(metrics.df[(metrics.df.case_endpoint.str.contains('localhost')) & 
                                                    (metrics.df.case_client_ip == ip) & (metrics.df.passed == 1)], aggregation), fig, ax)
        t+=1


In [ ]:
plt.scatter(metrics.df.date_int, metrics.df.case_duration, cmap='Blues', c=range(len(metrics.df.date_int)))

In [ ]:
def return_mean_duration_over_time(df):
    mean_time_v_endpoint = df.groupby('case_endpoint')
    mean_v_endpoint = mean_duration_v_case_endpoint.agg(aggregation)


In [ ]:
mean_time_v_endpoint = metrics.df.groupby('case_endpoint')
mean_v_endpoint = mean_time_v_endpoint.agg(aggregation)
mean_v_endpoint['ticks'] = pd.Series(range(len(mean_v_endpoint.index.values)), index=mean_v_endpoint.index)


In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.scatter(range(len(mean_v_endpoint.index.values)), mean_v_endpoint.case_duration.mean_duration)
ax.set_xticks(_graphs.create_uniform_tcks(mean_v_endpoint.index.values))
ax.set_xticklabels(mean_v_endpoint.index.values, rotation = 70)


In [ ]:
(len(metrics.df) - len(metrics.df[metrics.df.case_timestamp.isnull()])) / len(metrics.df)

In [ ]:
count_agg = {
    'scenario_uuid' : {
        'count': 'count'
    }
}


In [ ]:
def _get_case_count_by_service(df):
    service_names = df.case_service_name.unique()
    count_frame = df.groupby(['case_service_name', 'group_uuid']).agg({'scenario_uuid' : {'count': 'count'}})
    service_v_count = {}
    for service in service_names:
        serv_frame = count_frame.loc[service]
        max_scenario_uuid = serv_frame.scenario_uuid.max()
        largest_guiid = serv_frame.scenario_uuid[serv_frame.scenario_uuid == max_scenario_uuid].dropna().index.values
        largest_run = df[df.group_uuid ==  largest_guiid[0]]
        full_name = largest_run.case_service_name.str.cat(largest_run.case_id, sep='_')
        case_count = len(full_name.unique())
        service_v_count[service] = case_count
    return service_v_count

In [ ]:
def get_case_list_by_group(config):
#     Identity = namedtuple('Identity', ['service', 'id'])
    groups = config.get('groups')
    full_case_lists = {}
    for group_name, group in groups.items():
        cases = group['cases']
        if group.get('dependencies'):
            for dep in group.get('dependencies'):
                dependencies_tests = groups.get(dep).get('cases')
                cases +=  dependencies_tests
        full_case_lists[group_name] = cases
    return full_case_lists

In [ ]:
import json
with open('config.json', 'rb') as in_file:
    config = json.loads(in_file.read().decode())

In [ ]:
_get_case_count_by_service(metrics.df)

In [ ]:
count_frame = metrics.df.groupby(['case_service_name', 'group_uuid']).agg(count_agg)

In [ ]:
bolt = count_frame.loc['BOLTServicesBranch']

In [ ]:
biggest_guiid = bolt.scenario_uuid[bolt.scenario_uuid == bolt.scenario_uuid.max()].dropna().index[0]

In [ ]:
metrics.df[metrics.df.group_uuid == biggest_guiid].case_id.unique()

In [ ]:
count_frame.loc['BOLTServicesBranch'].scenario_uuid.describe().max().values

In [ ]:
count_frame.loc['BOLTServicesBranch'].scenario_uuid.max()

In [ ]:
count_frame.loc['BOLTServicesBranch'].scenario_uuid[count_frame.loc['BOLTServicesBranch'].scenario_uuid ==6276].dropna().index.values

In [18]:
i = 0
while i < 25:
    if i < 5:
        i += 1
        print('IN I < 5')
        continue
    print("i > 5")
    i += 2

IN I < 5
IN I < 5
IN I < 5
IN I < 5
IN I < 5
i > 5
i > 5
i > 5
i > 5
i > 5
i > 5
i > 5
i > 5
i > 5
i > 5
DUMPING TO CSV!!!!!!!ASLDKFJASL;KDJFLK;ASDJF


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/home/jshank/ext/code/venvz/anal/lib/python3.5/site-packages/pandas/indexes/base.py", line 2104, in get_loc
    return self._engine.get_loc(key)
  File "pandas/index.pyx", line 139, in pandas.index.IndexEngine.get_loc (pandas/index.c:4160)
  File "pandas/index.pyx", line 161, in pandas.index.IndexEngine.get_loc (pandas/index.c:4024)
  File "pandas/src/hashtable_class_helper.pxi", line 732, in pandas.hashtable.PyObjectHashTable.get_item (pandas/hashtable.c:13161)
  File "pandas/src/hashtable_class_helper.pxi", line 740, in pandas.hashtable.PyObjectHashTable.get_item (pandas/hashtable.c:13115)
KeyError: 'case_id'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/threading.py", line 1180, in run
    self.function(*self.args, **self.kwargs)